#  MentalQA Dataset Description

The full dataset consists of **500 annotated samples**. For the purpose of this shared task, we are using only the **training and development set**:

- **Train+Dev Set**: 350 samples  
- **Test Set**: 150 samples (withheld for evaluation purposes)

---

###  Dataset File: `MentalQA_500_data.csv` (tab-separated)

This dataset was recently accepted in the *IEEE ACCESS* journal.  
It contains **four columns** as follows:

1. **Question**  
2. **Answer**  
3. **Question Types** (`final_QT`)  
4. **Answer Strategies** (`final_AS`)

---

###  Question Type Labels (`final_QT`)

| Label | Name                       |Description |
|----------|----------------------------|----------------
| A        | **Diagnosis**              | Questions about clinical findings, tests, disease criteria, and manifestations. |
| B        | **Treatment**              | Questions about therapies, drug use, side effects, and contraindications. |
| C        | **Anatomy and Physiology** | Basic medical knowledge (e.g., tissues, organs, metabolism). |
| D        | **Epidemiology**           | Disease progression, prognosis, etiology, and risk factors. |
| E        | **Healthy Lifestyle**      | Diet, exercise, mood, and lifestyle factors affecting mental health. |
| F        | **Provider Choices**       | Recommendations for doctors, hospitals, or medical departments. |
| Z        | **Other**                  | General or uncategorized questions. |

---

###  Answer Strategy Labels (`final_AS`)

| Label   | Name              | Description |
|---------|-----------|-------------|
| 1       | **Information**    | Provides factual data, explanations, or resources. |
| 2       |  **Direct Guidance**| Offers suggestions, instructions, or behavioral advice. |
| 3       | **Emotional Support** | Gives reassurance, empathy, or emotional encouragement. |





GitHub: https://github.com/hasanhuz/MentalQA/tree/main

Paper: https://arxiv.org/abs/2405.12619

In [ ]:
!pip install openai==0.28
!pip install bert-score
!pip install -U langchain langchain-community
!pip install -U langchain-groq

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

In [ ]:
pd.set_option("display.max_colwidth", None, "display.max_rows", None)

In [ ]:
data = pd.read_csv('/content/Train_Dev.tsv', sep='\t')
data.sample(3)

### Start Cleaning Dataset
##  Data Preprocessing and Cleaning Overview

Before performing any analysis or model evaluation, it is crucial to clean and normalize the dataset to ensure consistency and quality.

This section outlines the preprocessing steps applied to the MentalQA Arabic dataset.

---

###  Cleaning Steps Applied:

1. **Displayed full text columns** – Enabled full visibility of long question/answer text using `pd.set_option("display.max_colwidth", None)`.
2. **Email Removal** – Removed any embedded email addresses using regular expressions.
3. **Separation Rules** – Inserted spaces between:
   - Arabic letters and numbers (e.g., "دواء20" → "دواء 20")
   - Arabic and English terms (e.g., "علاجmg" → "علاج mg")
   - Numbers and English words (e.g., "mg75" → "mg 75")
4. **Punctuation Cleanup** – Removed all punctuation marks except the Arabic question mark "؟".
5. **Arabic Letter Normalization** – Standardized variations of alef (ا), ta marbuta (ه), and others.
6. **Numeric Normalization** – Converted Arabic-Indic numerals (٠١٢...) to Western digits (012...).
7. **Repetition Reduction** – Compressed elongated words (e.g., "رااااائع" → "رائع").
8. **Question Mark Spacing** – Ensured space before and after "؟" to avoid it sticking to adjacent words.
9. **Custom Corrections** – Applied multiple custom dictionaries to fix common spelling errors, typos, and fused terms (e.g., "ماا" → "ما", "بلنتحار" → "بالانتحار").

---

###  Word-Level Error Detection via Frequency Analysis

To further enhance text quality, we analyzed both the cleaned questions and answers at the word level:

- Combined all tokens from `question_clean_1` and `answer_clean_1`.
- Used `collections.Counter` to calculate word frequencies.
- Created a sorted DataFrame to highlight rare and suspicious words.
- Manually reviewed low-frequency words to detect:
  - Typos (e.g., "هاذا", "معايا", "مفارقهمتصل")
  - Word merges and informal language
- Built custom correction dictionaries based on the findings to normalize the vocabulary.

This iterative refinement helped reduce vocabulary noise and improve input quality for downstream modeling.


In [ ]:
data.columns

In [ ]:
data.shape

In [ ]:
df = data[['question', 'answer']]
df

In [ ]:
import re

def clean_arabic_text(text):
    if pd.isnull(text):
        return ""
    # 0. Normalize numbers
    text = text.translate(str.maketrans("٠١٢٣٤٥٦٧٨٩", "0123456789"))

    # 1. Remove Email (واستبداله بمسافة)
    text = re.sub(r'\S*@\S+', ' ', text)

    # 2. Make Distance between Arabic and Numbers/English
    text = re.sub(r'([ء-ي])(\d+)', r'\1 \2', text)
    text = re.sub(r'(\d+)([ء-ي])', r'\1 \2', text)
    text = re.sub(r'([ء-ي])([a-zA-Z]+)', r'\1 \2', text)
    text = re.sub(r'([a-zA-Z]+)([ء-ي])', r'\1 \2', text)
    text = re.sub(r'([a-zA-Z]+)(\d+)', r'\1 \2', text)
    text = re.sub(r'(\d+)([a-zA-Z]+)', r'\1 \2', text)

    # 3. Remove repeated characters (مثلاً: كييييف → كيف)
    text = re.sub(r'(.)\1{2,}', r'\1', text)

    # 4. Normalize Arabic letters
    text = re.sub(r'[إأآا]', 'ا', text)
    text = re.sub(r'ة', 'ه', text)
    text = re.sub(r'گ', 'ك', text)

    # 5. Remove punctuation (except ؟) مع استبدالها بمسافة
    text = re.sub(r'[^\w\s؟]', ' ', text)

    # 6. Ensure space before and after question mark
    text = re.sub(r'\s*؟\s*', ' ؟ ', text)

    # 7. Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

df["question_clean_1"] = df["question"].apply(clean_arabic_text)
df["answer_clean_1"] = df["answer"].apply(clean_arabic_text)

In [ ]:
all_text = ' '.join(df['question_clean_1'].astype(str))
words = all_text.split()
word_freq = Counter(words)
freq_df = pd.DataFrame(word_freq.items(), columns=['word', 'count']).sort_values(by='count', ascending=True)
freq_df

In [ ]:
def apply_custom_corrections(text):
    corrections = {
        "قي": "في",
        "انااذكر": "أنا أذكر",
        "ماا": "ما",
        "عليكماعاني": "عليكم اعاني",
        "هده": "هذه",
        "للاكتئا": "للاكتئاب",
        "بلسوء": "ب السوء",
        "ءاذي": "الذي",
        "ماكدر": "ما اقدر",
        "معايا": "معي",
        "حتا": "حتى",
        "انوا": "أنه",
        "اخري": "أخرى",
        "بلنتحار": "ب الانتحار",
        "بلاحباط": "ب الإحباط",
        "منهاا": "منها",
        "ابغا": "أريد",
        "طمانتني": "طمنتني",
        "وواحبها": "واحبها",
        "مفعولخ": "مفعوله",

    }

    for wrong, right in corrections.items():
        text = text.replace(wrong, right)
    return text

df["question_clean_2"] = df["question_clean_1"].apply(apply_custom_corrections)
#df["answer_clean_2"] = df["answer_clean_1"].apply(clean_arabic_text)

In [ ]:
all_text_ = ' '.join(df['answer_clean_1'].astype(str))
words_ = all_text_.split()
word_freq_ = Counter(words_)
freq_df_ = pd.DataFrame(word_freq_.items(), columns=['word', 'count']).sort_values(by='count', ascending=True)
freq_df_

In [ ]:
def apply_answer_corrections(text):
    corrections = {
        "النفسيوربما": "النفسي وربما",
        "العقاقيرويستحسن": "العقاقير ويستحسن",
        "وضروريوايذاء": "و ضروري و إيذاء",
        "والجرعهليقدرها": "و الجرعة ليقدرها",
        "تسوءوربما": "تسوء وربما",
        "تغييراو": "تغيير أو",
        "ابق": "أبقى",
        "الادويهنحتاج": "الأدوية نحتاج",
        "الاعتماديهالاعتماد": "الاعتمادية الاعتماد",
        "الحلتدريجيا": "الحل تدريجا",
        "مفارقهالتعلق": "مفارقة التعلق",
        "خطيرهلانها": "خطيرة لأنها",
        "نفسيالافكاراالانتحاريه": "نفسي الأفكار الانتحارية",
        "الخضارقللي": "الخضار قللي",
        "غلى": "على",
        "الجتماعي": "الاجتماعي",
        "لكليهماونتوقف": "لكليهما و نتوقف",
        "وانواعهاولوم": "و أنواعها و لوم",
        "الناسنحتاج": "الناس نحتاج",
        "المراءه": "المراة",
        "هاذا": "هذا",
        "المسسبه": "المسببه",
        "اى": "اي",
        "عضوى": "عضوي",
        "النفسانيالافكار": "النفساني الأفكار",
        "هدا": "هذا",
        "اسرغ": "اسرع",
        "والاسترخاءوهذا": "و الاسترخاء و هذا",
        "الاخرفتصل": "الاخر فتصل",
        "الاخروعندما": "الاخر و عندما",
        "دواءي": "دوائي",
        "مشهورهاسمها": "مشهور اسمه"
    }

    for wrong, right in corrections.items():
        text = text.replace(wrong, right)
    return text

df["answer_clean_2"] = df["answer_clean_1"].apply(apply_answer_corrections)

In [ ]:
# Cleaning Columns
df[['question_clean_2', 'answer_clean_2']]

In [ ]:
GROQ_API_KEY="gsk_...."
from langchain_groq import ChatGroq
from langchain.schema import HumanMessage


chat = ChatGroq(
    groq_api_key=GROQ_API_KEY,
    model_name="llama3-70b-8192",
    temperature=0.1,
    max_tokens=1024
)

In [ ]:
def build_zero_shot_prompt(question):
    return f"أجب على السؤال التالي باحترافية وباللغة العربية:\n\nالسؤال: {question}\n\nالإجابة:"

In [ ]:
def generate_llama_answer(question):
    prompt = build_zero_shot_prompt(question)
    response = chat.invoke([
        HumanMessage(content=prompt)
    ])
    return response.content.strip()


In [ ]:
sample_df = df[['question_clean_2', 'answer_clean_2']].sample(n=10, random_state=42).copy()
sample_df['llama_zero_shot_answer'] = sample_df['question_clean_2'].apply(generate_llama_answer)
sample_df.to_csv("llama3_zero_shot_generated_answers.csv", index=False)

In [ ]:
from bert_score import score

P, R, F1 = score(
    sample_df['llama_zero_shot_answer'].tolist(),
    sample_df['answer_clean_2'].tolist(),
    lang="ar",
    verbose=True
)

sample_df['bertscore_f1'] = F1.tolist()
sample_df.to_csv("llama3_zero_shot_generated_answers.csv", index=False)
print(f"✅ Average BERTScore F1: {F1.mean().item():.2f}")

In [ ]:
example_1 = {
    "question": "بقالي اكتر من اسبوعين مش بنام ولا باكل و لو نمت ساعه واحده في اليوم افضل صاحيه 3 ايام و بحلم بكابوس بيتكرر علي طول اني باذى نفسي لدرجه اني فكرت في كده	",
    "answer": "أنتي بحاجه لزيارة طبيب نفسي للتقيم اخذ السيره المرضيه بشكل ادق ولمعرفة تفاصيل اخرى مهمه في التشخيص لا تترددي في الاستشاره لان علامات المرض النفسي الحاد بدات تظهر بقله الحاجه للنوم تمنياتي بالشفاء العاجل	"
}

example_2 = {
    "question": "تمنعني ظروف ما اقدر اروح دكتور اخصائي ولا اشرح حالتي كدامه صرت افكر بالانتحار بسبب هاي الحاله شي مخيف أشد أنواع العذاب لدرجه الموت أهون الموت أهون	",
    "answer": "السلام عليكم علاج الادمان محتاج طبيب وادوية والاهم يكون عندك اراده قويه للتخلص من هذه السموم وعلى قدر عزيمتك هيكون التوفيق لنجاح العلاج ساعد نفسك	"
}

example_3 = {
    "question": "اعاني من الخوف وتنبؤات من كلام اهلي وبصمتهم انهم يبون يقتلوني بدون سبب والي محرضهم ابوي",
    "answer": "التقلبات المزاجيه في سنك المبكره كثيره ومن المفيد مراجعه طبيب للتشخيص الدقيق	"
}


In [ ]:
def build_fewshot_prompt(new_question):
    prompt = f"""
أجب على الأسئلة التالية باحترافية وباللغة العربية:

س: {example_1['question']}
ج: {example_1['answer']}

س: {example_2['question']}
ج: {example_2['answer']}

س: {example_3['question']}
ج: {example_3['answer']}

س: {new_question}
ج:"""
    return prompt.strip()

In [ ]:
def generate_llama_answer(question):
    prompt = build_fewshot_prompt(question)
    response = chat.invoke([
        HumanMessage(content=prompt)
    ])
    return response.content.strip()


In [ ]:
sample_df = df[['question_clean_2', 'answer_clean_2']].sample(n=10, random_state=42).copy()
sample_df['llama_few_shot_answer'] = sample_df['question_clean_2'].apply(generate_llama_answer)
sample_df.to_csv("llama_few_shot_answer.csv", index=False)

In [ ]:
from bert_score import score

P, R, F1 = score(
    sample_df['llama_few_shot_answer'].tolist(),
    sample_df['answer_clean_2'].tolist(),
    lang="ar",
    verbose=True
)

sample_df['bertscore_f1'] = F1.tolist()
sample_df.to_csv("llama_few_shot_answer.csv", index=False)
print(f"✅ Average BERTScore F1: {F1.mean().item():.2f}")

In [ ]:
import random

def generate_fewshot_sc_answer(question, n=10): # I traied 5 answers then 10 anwsers but the F1 is same
    answers = []
    for _ in range(n):
        prompt = build_fewshot_prompt(question)
        response = chat.invoke(prompt, temperature=0.1)
        answer = response.content.strip()
        answers.append(answer)

    most_common = Counter(answers).most_common(1)[0][0]
    return most_common

In [ ]:
sample_df = df[['question_clean_2', 'answer_clean_2']].sample(n=10, random_state=42).copy()
sample_df['llama_fewshot_sc_answer'] = sample_df['question_clean_2'].apply(generate_fewshot_sc_answer)
sample_df.to_csv("llama_fewshot_sc_answer.csv", index=False)

In [ ]:
from bert_score import score

P, R, F1 = score(
    sample_df['llama_fewshot_sc_answer'].tolist(),
    sample_df['answer_clean_2'].tolist(),
    lang="ar",
    verbose=True
)

sample_df['bertscore_f1'] = F1.tolist()
sample_df.to_csv("llama_fewshot_sc_answer.csv", index=False)
print(f"✅ Average BERTScore F1: {F1.mean().item():.2f}")

In [ ]:
def generate_multiple_llama_answers(question, n=5):
    answers = []
    for _ in range(n):
        prompt = build_fewshot_prompt(question)
        response = chat.invoke(prompt, temperature=0.1)
        answers.append(response.content.strip())
    return answers

In [ ]:
def build_refinement_prompt(question, answers):
    joined_answers = "\n\n".join(
    [f"Answer {i+1}: {ans}" for i, ans in enumerate(answers)]
    )

    return f"""Question: {question}

    These are different answers to the same question:

    {joined_answers}

    Based on the previous answers, write an improved and ideal answer, taking into account accuracy and clarity:

Final answer:"""

In [ ]:
sample_df = df[['question_clean_2', 'answer_clean_2']].sample(n=10, random_state=42).copy()
sample_df['llama_er_answer'] = sample_df['question_clean_2'].apply(
    lambda q: build_refinement_prompt(q, generate_multiple_llama_answers(q, n=5))
)
sample_df.to_csv("llama_er_answer.csv", index=False)

In [ ]:
from bert_score import score

P, R, F1 = score(
    sample_df['llama_er_answer'].tolist(),
    sample_df['answer_clean_2'].tolist(),
    lang="ar",
    verbose=True
)

sample_df['bertscore_er_answer'] = F1.tolist()
sample_df.to_csv("llama_er_answer.csv", index=False)
print(f"✅ Average BERTScore F1: {F1.mean().item():.2f}")

In [ ]:
example_1 = {
    "question": "بقالي اكتر من اسبوعين مش بنام ولا باكل و لو نمت ساعه واحده في اليوم افضل صاحيه 3 ايام و بحلم بكابوس بيتكرر علي طول اني باذى نفسي لدرجه اني فكرت في كده	",
    "answer": "أنتي بحاجه لزيارة طبيب نفسي للتقيم اخذ السيره المرضيه بشكل ادق ولمعرفة تفاصيل اخرى مهمه في التشخيص لا تترددي في الاستشاره لان علامات المرض النفسي الحاد بدات تظهر بقله الحاجه للنوم تمنياتي بالشفاء العاجل	"
}

example_2 = {
    "question": "تمنعني ظروف ما اقدر اروح دكتور اخصائي ولا اشرح حالتي كدامه صرت افكر بالانتحار بسبب هاي الحاله شي مخيف أشد أنواع العذاب لدرجه الموت أهون الموت أهون	",
    "answer": "السلام عليكم علاج الادمان محتاج طبيب وادوية والاهم يكون عندك اراده قويه للتخلص من هذه السموم وعلى قدر عزيمتك هيكون التوفيق لنجاح العلاج ساعد نفسك	"
}

example_3 = {
    "question": "اعاني من الخوف وتنبؤات من كلام اهلي وبصمتهم انهم يبون يقتلوني بدون سبب والي محرضهم ابوي",
    "answer": "التقلبات المزاجيه في سنك المبكره كثيره ومن المفيد مراجعه طبيب للتشخيص الدقيق	"
}

example_4 = {
    "question": "عمرها 15 منذ قرابه شهرين او اكثر بدات حالتها تتغير من الهدوء الى العصبيه وعدم النوم",
    "answer": "هناك العديد من الاسئله هل يوجد خلافات في المنزل هل تعرضت لاي ضغوط او اي صدمه ؟ يمكنك التواصل مباشره مع معالج نفسي للمساعده"
}

example_5 = {
    "question": "قلق وتوتر ووسواس قهري وتاتاه في الكلام وعجز داخلي اظن اني مكروه واني لا اقدر افعل شي	",
    "answer": "انشغل بالدراسه اوالعمل او الانشطه الاجتماعيه ومع تلقي العلاج النفسي تتحسن الحاله مع التدريب والصبر"
}


In [ ]:
def build_fewshot_5_prompt(new_question):
    prompt = f"""
أجب على الأسئلة التالية باحترافية وباللغة العربية:

س: {example_1['question']}
ج: {example_1['answer']}

س: {example_2['question']}
ج: {example_2['answer']}

س: {example_3['question']}
ج: {example_3['answer']}

س: {example_4['question']}
ج: {example_4['answer']}

س: {example_5['question']}
ج: {example_5['answer']}

س: {new_question}
ج:"""
    return prompt.strip()

In [ ]:
def generate_llama_fewshot_5_answer(question):
    prompt = build_fewshot_5_prompt(question)
    response = chat.invoke([
        HumanMessage(content=prompt)
    ])
    return response.content.strip()

In [ ]:
sample_df = df[['question_clean_2', 'answer_clean_2']].sample(n=10, random_state=42).copy()
sample_df['llama_fewshot_5_answer'] = sample_df['question_clean_2'].apply(generate_llama_fewshot_5_answer)
sample_df.to_csv("llama_fewshot_5_answer.csv", index=False)

In [ ]:
from bert_score import score

P, R, F1 = score(
    sample_df['llama_fewshot_5_answer'].tolist(),
    sample_df['answer_clean_2'].tolist(),
    lang="ar",
    verbose=True
)

sample_df['bertscore_fewshot_5'] = F1.tolist()
sample_df.to_csv("llama_fewshot_5_answer.csv", index=False)
print(f"✅ 5 - Few-Shot BERTScore F1: {F1.mean().item():.2f}")

In [ ]:
def build_fewshot_cot_prompt(new_question):
    prompt = f"""

Question: I can't sleep and feel stressed all day. Do I have a mental illness?

Let's analyze the condition gradually:
1. Look at the symptoms: insomnia (inability to sleep) and constant stress.
2. Associated with: disorders such as generalized anxiety or depression.
3. To properly diagnose the condition: You must know a few things.
4. To answer: Provide a professional answer that takes into account the above factors using one or more of the strategies:
- providing information
- Direct answers
- Emotional support

Answer: It is best to consult a psychologist to evaluate the condition. Lifestyle changes or behavioral therapy may help alleviate the symptoms.

Make sure your answer is fully formatted in Arabic and specifically designed to diagnose mental health conditions

Question: {new_question}
Answer:"""
    return prompt.strip()

In [ ]:
def generate_llama_fewshot_cot_answer(question):
    prompt = build_fewshot_cot_prompt(question)
    response = chat.invoke([
        HumanMessage(content=prompt)
    ])
    return response.content.strip()

In [ ]:
sample_df = df[['question_clean_2', 'answer_clean_2']].sample(n=10, random_state=42).copy()
sample_df['llama_fewshot_cot_answer'] = sample_df['question_clean_2'].apply(generate_llama_fewshot_cot_answer)
sample_df.to_csv("llama_fewshot_cot_answer.csv", index=False)

In [ ]:
from bert_score import score

P, R, F1 = score(
    sample_df['llama_fewshot_cot_answer'].tolist(),
    sample_df['answer_clean_2'].tolist(),
    lang="ar",
    verbose=True
)

sample_df['bertscore_llama_fewshot_cot_answer'] = F1.tolist()
sample_df.to_csv("llama_fewshot_cot_answer.csv", index=False)
print(f"✅ CoT BERTScore F1: {F1.mean().item():.2f}")

In [ ]:
!pip install -U google-generativeai

In [ ]:
from huggingface_hub import login
login()  # أدخل توكن HF

In [ ]:
import google.generativeai as genai
print(f"Google Generative AI version: {genai.__version__}")

In [ ]:
from huggingface_hub import __version__ as hf_version
print(f"Hugging Face Hub version: {hf_version}")

In [ ]:
import sys
print(f"Python version: {sys.version}")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Allam
allam_model = AutoModelForCausalLM.from_pretrained("ALLaM-AI/ALLaM-7B-Instruct-preview", torch_dtype="auto", device_map="auto")
allam_tokenizer = AutoTokenizer.from_pretrained("ALLaM-AI/ALLaM-7B-Instruct-preview")

# Qwen
qwen_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-7B-Instruct", torch_dtype="auto", device_map="auto")
qwen_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")

In [ ]:
# FINAL aLLAm
def gen_hf(model, tokenizer, prompt):
    # 1. Tokenize the prompt and move to model's device
    toks = tokenizer(prompt, return_tensors="pt").to(model.device)

    # 2. Generate the model output
    out = model.generate(**toks, max_new_tokens=1024)

    # 3. Decode only the generated part (exclude input prompt)
    generated_tokens = out[0][toks["input_ids"].shape[1]:]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    return response


In [ ]:
# FINAL Qwen
def gen_qwen(model, tokenizer, prompt):
    # 1. Tokenize the prompt and move to model's device
    toks = tokenizer(prompt, return_tensors="pt").to(model.device)

    # 2. Generate output
    out = model.generate(**toks, max_new_tokens=1024)

    # 3. Decode only the generated part
    generated_tokens = out[0][toks["input_ids"].shape[1]:]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    return response

In [ ]:
# FINAL Gemini
import google.generativeai as genai
genai.configure(api_key="..")
gemini_model = genai.GenerativeModel("gemini-2.5-flash")

def gen_gemini(model, prompt):
    response = model.generate_content(prompt)
    return response.text


In [ ]:
#FINAL GPT

import openai

openai.api_key = "sk-proj...."
openai_model = "gpt-4o"

def gen_openai(prompt, model_name):
    response = openai.ChatCompletion.create(
        model=model_name,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [ ]:
# FINAL

import pandas as pd
from tqdm import tqdm
from bert_score import score

# Load 15 samples
#df = pd.read_csv("Train_Dev.tsv", sep="\t")
df_sample = df[['question_clean_2', 'answer_clean_2']].dropna().sample(n=20, random_state=42).reset_index(drop=True)

# Prepare all prompts
df_sample["prompt"] = df_sample["question_clean_2"].apply(build_fewshot_prompt)

# Initialize result containers
results = {
    "allam": [],
    "qwen": [],
    "gpt": [],
    "gemini": []
}


In [ ]:
#FINAL

# Allam
for prompt in tqdm(df_sample["prompt"], desc="Allam"):
    ans = gen_hf(allam_model, allam_tokenizer, prompt)
    results["allam"].append(ans)

# Qwen
for prompt in tqdm(df_sample["prompt"], desc="Qwen"):
    ans = gen_qwen(qwen_model, qwen_tokenizer, prompt)
    results["qwen"].append(ans)

# OpenAI GPT
for prompt in tqdm(df_sample["prompt"], desc="GPT"):
    ans = gen_openai(prompt, openai_model)
    results["gpt"].append(ans)

# Gemini
for prompt in tqdm(df_sample["prompt"], desc="Gemini"):
    ans = gen_gemini(gemini_model, prompt)
    results["gemini"].append(ans)


In [ ]:
bert_results = {}

for model_name in results.keys():
    print(f"Scoring {model_name}...")
    P, R, F1 = score(
        results[model_name],
        df_sample["answer_clean_2"].tolist(),
        lang="ar",
        verbose=True
    )
    bert_results[model_name] = {
        "precision": P.mean().item(),
        "recall": R.mean().item(),
        "f1": F1.mean().item()
    }


In [ ]:
print("\n📊 Final Average BERTScore F1 per Model:")
for model_name, scores in bert_results.items():
    print(f"{model_name}: F1 = {scores['f1']:.4f}")